In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, Dropdown
import matplotlib.ticker as mticker

# Sample DataFrame (replace with your actual dataset)
data = pd.read_csv('data.csv')  # Ensure df is loaded with your data

# Ensure the 'date' column is in datetime format
data['date'] = pd.to_datetime(data['date'])
data['weekday'] = data['date'].dt.weekday
data['hour'] = data['date'].dt.hour

# Create a dictionary mapping zone_id to zone name for use in the title
zone_name_map = data[['zone_id', 'name']].drop_duplicates().set_index('zone_id')['name'].to_dict()

# Define function to plot hourly impressions
def plot_impressions(zone_id, date, prev_date):
    # Filter data for selected zone and dates
    selected_data = data[data['zone_id'] == zone_id]
    date_data = selected_data[selected_data['date'].dt.date == pd.to_datetime(date).date()]
    prev_date_data = selected_data[selected_data['date'].dt.date == pd.to_datetime(prev_date).date()]
    
    # Error handling for empty datasets
    if date_data.empty or prev_date_data.empty:
        print(f"No data available for Zone ID {zone_id} on the selected dates.")
        return

    # Group by hour to sum impressions
    hourly_date = date_data.groupby('hour')['impressions'].sum()
    hourly_prev_date = prev_date_data.groupby('hour')['impressions'].sum()

    # Plot the impressions
    plt.figure(figsize=(10, 6))
    plt.plot(hourly_date.index, hourly_date.values, label=date, marker='o')
    plt.plot(hourly_prev_date.index, hourly_prev_date.values, label=prev_date, marker='o')
    plt.title(f'Hourly Total Impressions for Zone ID {zone_id} - {zone_name_map.get(zone_id, "Unknown")}')
    plt.xlabel('Hour of Day')
    plt.ylabel('Total Impressions')
    plt.legend()
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.xticks(range(24), labels=[f'{i}:00' for i in range(24)], rotation=45)
    
    # Format Y-axis as human-readable (e.g., 1K, 10K)
    ax = plt.gca()
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x / 1e3:.0f}K'))
    
    plt.tight_layout()
    plt.show()

# Dropdown options for zone_id and available dates
zone_options = sorted(data['zone_id'].unique())
date_options = sorted(data['date'].dt.date.unique())

# Interactive dropdown menus with default values
interact(
    plot_impressions,
    zone_id=Dropdown(options=zone_options, value=zone_options[0], description="Zone ID"),
    date=Dropdown(options=date_options, value=date_options[-1], description="Date"),
    prev_date=Dropdown(options=date_options, value=date_options[-2], description="Previous Date")
)
